### Draft Working SQL code for ACID properties

In [ ]:
BEGIN TRANSACTION;

-- Step 1: Insert a new record in the Orders table
INSERT INTO Orders (
    CustomerID,
    OrderDate,
    TotalAmount
)
VALUES (
    @CustomerID,
    GETDATE(),
    @TotalAmount
);

-- Step 2: Add each item from the order into the OrderDetails table
INSERT INTO OrderDetails (
    OrderID,
    ProductID,
    Quantity
)
SELECT 
    SCOPE_IDENTITY(), -- Get the ID of the newly inserted order
    p.ProductID,
    oi.Quantity
FROM 
    ShoppingCartItems oi
JOIN 
    Products p ON oi.ProductID = p.ProductID;

-- Step 3: Decrease the quantity in stock for each item in the Products table
UPDATE Products
SET Quantity = Quantity - oi.Quantity
FROM 
    ShoppingCartItems oi
WHERE 
    oi.ProductID = Products.ProductID;

-- Commit the transaction if all steps succeed
IF @@ERROR = 0
BEGIN
    COMMIT TRANSACTION;
END
ELSE
BEGIN
    ROLLBACK TRANSACTION;
END;

-- Return the OrderID as confirmation
SELECT SCOPE_IDENTITY() AS OrderID;

### OR

In [ ]:
BEGIN TRANSACTION;

-- Step 1: Insert the new order into the Orders table
INSERT INTO Orders (customer_id, order_date, total)
VALUES (@customer_id, NOW(), @order_total);

-- Get the newly generated order_id
SET @order_id = LAST_INSERT_ID();

-- Step 2: Add items into the OrderDetails table and update the Products table
-- This loop assumes that @order_items is an array or a set of order items passed to the transaction
FOR EACH @item IN @order_items DO
    -- Insert the order details (items ordered)
    INSERT INTO OrderDetails (order_id, product_id, quantity, unit_price)
    VALUES (@order_id, @item.product_id, @item.quantity, @item.unit_price);

    -- Step 3: Decrease the stock in the Products table for each item ordered
    UPDATE Products
    SET stock_quantity = stock_quantity - @item.quantity
    WHERE product_id = @item.product_id;

    -- Check if there are enough stock for the item, if not, rollback the transaction
    IF (SELECT stock_quantity FROM Products WHERE product_id = @item.product_id) < 0 THEN
        ROLLBACK TRANSACTION;
        -- Handle insufficient stock case (e.g., show an error to the user)
        RAISE ERROR 'Not enough stock for product ID ' + @item.product_id;
        RETURN;
    END IF;
END FOR;

-- Step 4: Commit the transaction if everything is successful
COMMIT TRANSACTION;
